In [ ]:
# default_exp losses

In [ ]:
# export
import torch
import torch.nn as nn
import torch.nn.functional as F

# Losses

> A selection of loss functions I've used with this dataset. These are research code with no guarantees of correctness or usefulness.

## Axis-Angle Cosine Distance

> Cosine similarity between rotation vectors.

This is probably not a correct distance metric for rotations. [This blog][belousov] makes an argument that there's only one correct distance metric for rotations and it's the angular distance between unit quaternions. But, it is quick to evaluate on rotation vectors.

[belousov]: http://www.boris-belousov.net/2016/12/01/quat-dist/

In [ ]:
# exports
def aa_cosine(out, target):
    if out.ndim == 2:
        b, d = out.size()
    elif out.ndim == 3:
        b, f, d = out.size()
        assert f == 1, f'{out.size()}'
    j = d//3
    out, target = out.view(b, j, 3), target.view(b, j, 3)
    def theta(x, eps=1e-6):
        return torch.sqrt(torch.clamp(torch.sum(x**2, 2, keepdims=True), eps, 2*math.pi))
    theta_a = theta(out)
    theta_b = theta(target)
    cosine_sim = F.cosine_similarity(out, target, dim=2)
    cosine_sim_loss = 1. - cosine_sim
    cosine_angle_diff = 1. - torch.cos(theta_a - theta_b)
    return torch.mean(cosine_sim_loss + cosine_angle_diff[:,:,0])

## VPoser

> The loss function used by the VPoser VAE in the [SMPL-X][smplx] paper.

[smplx]: https://ps.is.tuebingen.mpg.de/uploads_file/attachment/attachment/497/SMPL-X.pdf

In [ ]:
# exports
class GeodesicLossR(nn.Module):
    def __init__(self, reduction='batchmean'):
        super(geodesic_loss_R, self).__init__()

        self.reduction = reduction
        self.eps = 1e-6

    # batch geodesic loss for rotation matrices
    def bgdR(self,m1,m2):
        batch = m1.shape[0]
        m = torch.bmm(m1, m2.transpose(1, 2))  # batch*3*3

        cos = (m[:, 0, 0] + m[:, 1, 1] + m[:, 2, 2] - 1) / 2
        cos = torch.min(cos, m1.new(np.ones(batch)))
        cos = torch.max(cos, m1.new(np.ones(batch)) * -1)

        return torch.acos(cos)

    def forward(self, ypred, ytrue):
        theta = self.bgdR(ypred,ytrue)
        if self.reduction == 'mean':
            return torch.mean(theta)
        if self.reduction == 'batchmean':
            return torch.mean(torch.sum(theta, dim=theta.shape[1:]))
        else:
            return theta

def vposer_likelihood(
    self,
    dec_out,
    pose_target,
    body_model,
    loss_kl_wt=torch.tensor(0.005),
    loss_rec_wt=torch.tensor(4),
    loss_matrot_wt=torch.tensor(2),
    loss_jtr_wt=torch.tensor(2),
    callback=None
):
    """
    Default settings for loss weights taken from:
    https://github.com/nghorbani/human_body_prior/blob/master/src/human_body_prior/train/V02_05/V02_05.yaml
    """
    l1_loss = torch.nn.L1Loss(reduction="mean")
    geodesic_loss = GeodesicLossR(reduction="mean")

    # cast decoder output to aa
    bs, f, d = pose_target.size()
    assert f == 1, 'assuming only one frame'
    pose_target = pose_target.squeeze()
    pose_out_matrot = dec_out.view(-1, 3, 3)
    pose_out = matrot2aa(pose_out_matrot).view(bs, -1, 3)        

    # Reconstruction loss - L1 on the output mesh
    with torch.no_grad():
        bm_orig = body_model(pose_body=pose_target)
    bm_rec = body_model(pose_body=pose_out.contiguous().view(bs, -1))
    v2v = l1_loss(bm_rec.v, bm_orig.v)

    # Geodesic loss between rotation matrices
    matrot_loss = loss_matrot_wt * geodesic_loss(
        pose_out_matrot, aa2matrot(pose_target.view(-1, 3))
    )
    # L1 Loss on joint positions
    jtr_loss = l1_loss(bm_rec.Jtr, bm_orig.Jtr)

    # apply weights to make weighted loss
    weighted_loss = (
        loss_matrot_wt * matrot_loss + loss_rec_wt * v2v + loss_jtr_wt * jtr_loss
    )

    # log results
    with torch.no_grad():
        unweighted_loss = matrot_loss + v2v + jtr_loss
        if callback is not None:
            callback(all_univariate_tensors_in(locals()))

    return weighted_loss

## Discretized Euler Angles

> Intuitively, an ADC applied to Euler angles. Allows use of a NLL loss, which is can be useful for training a neural network.

In [ ]:
#exports
def discretize(x, nquant, eps=1e-6, dither=False, zero_centered=True):
    if zero_centered:
        x = x + math.pi
    m = math.pi*2
    assert x.max() < m
    x = x/m # scale to between zero and 1
    x = x*nquant
    if dither:
        d = 2.*(torch.rand_like(x)-0.5)
        x = torch.clamp(x+d, 0, nquant-eps)
    return torch.floor(x).long() # bin account to nquant levels

class DiscretizedEulerLoss(nn.Module):
    def __init__(self, nquant, dither=False, zero_centered=True):
        super().__init__()
        self.nquant, self.dither, self.zero_centered = nquant, dither, zero_centered

    def forward(self, out, target):
        assert out.size(-1) == self.nquant, f'trailing dimension should hold logits {out.size()}'
        target = discretize(target, self.nquant, dither=self.dither, zero_centered=self.zero_centered)
        return F.nll_loss(out.view(-1, self.nquant), target.view(-1))